## Model training and tuning for the financials sector

In [1]:
import pandas as pd
import numpy as np

import time

from darts import TimeSeries, concatenate
from darts.utils.model_selection import train_test_split
from darts.models.forecasting.xgboost import XGBModel
from darts.dataprocessing.transformers.scaler import Scaler
from darts.metrics.metrics import mape, mae
from darts.models.forecasting.prophet_model import Prophet
from darts.models.forecasting.sf_auto_theta import AutoTheta
from darts.models.forecasting.linear_regression_model import LinearRegressionModel
from darts.dataprocessing.transformers import StaticCovariatesTransformer
from darts.dataprocessing.transformers.boxcox import BoxCox

Importing plotly failed. Interactive plots will not work.
d:\investing-bot\.venv\Lib\site-packages\fs\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [2]:
df = pd.read_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')
df['date'] = pd.to_datetime(df['date'])

df_financials = df[df['GICS Sector'] == 'Financials'].sort_values(by=['ticker', 'date']).reset_index(drop=True)

In [30]:
df['GICS Sector'].value_counts()

GICS Sector
Industrials               193459
Financials                190485
Information Technology    177149
Health Care               151348
Consumer Discretionary    130185
Consumer Staples           96649
Real Estate                79757
Utilities                  78786
Materials                  63394
Energy                     58523
Communication Services     58017
Name: count, dtype: int64

In [3]:
df_financials['ticker'].unique()

array(['ACGL', 'AFL', 'AIG', 'AIZ', 'AJG', 'ALL', 'AMP', 'AON', 'APO',
       'AXP', 'BAC', 'BEN', 'BK', 'BLK', 'BRK-B', 'BRO', 'BX', 'C', 'CB',
       'CBOE', 'CFG', 'CINF', 'CME', 'COF', 'CPAY', 'DFS', 'EG', 'ERIE',
       'FDS', 'FI', 'FIS', 'FITB', 'GL', 'GPN', 'GS', 'HBAN', 'HIG',
       'ICE', 'IVZ', 'JKHY', 'JPM', 'KEY', 'KKR', 'L', 'MA', 'MCO', 'MET',
       'MKTX', 'MMC', 'MS', 'MSCI', 'MTB', 'NDAQ', 'NTRS', 'PFG', 'PGR',
       'PNC', 'PRU', 'PYPL', 'RF', 'RJF', 'SCHW', 'SPGI', 'STT', 'SYF',
       'TFC', 'TROW', 'TRV', 'USB', 'V', 'WFC', 'WRB', 'WTW'],
      dtype=object)

In [4]:
data = {}
static_covs = pd.DataFrame({'ticker': df_financials['ticker'].unique()})

for ticker, group in df_financials.groupby('ticker'):
    series = TimeSeries.from_dataframe(
        group,
        time_col='date',
        value_cols=['close', 'high', 'low', 'open', 'volume'],
        freq='B',
        static_covariates=pd.Series(ticker)
    )
    data[ticker] = series

In [5]:
# encode each ticker as a static covariate
transformer = StaticCovariatesTransformer()
ts = [d for d in data.values()]
transformer.fit(ts)

for ticker, series in data.items():
    data[ticker] = transformer.transform(series)

In [6]:
train = {}
test = {}

for ticker, series in data.items():
    train[ticker], test[ticker] = train_test_split(series, test_size=0.1)
    last_train = train[ticker][-1:]
    test_with_context = last_train.append(test[ticker]) # append last point of train to test as our forecast requires it
    test[ticker] = test_with_context

## Gradient Boosting

### Train a local model first

In [7]:
models = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = XGBModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models[ticker] = model

Training model for ACGL
Model for ACGL trained in 0.30 seconds
**************************************************
Training model for AFL
Model for AFL trained in 0.33 seconds
**************************************************
Training model for AIG
Model for AIG trained in 0.34 seconds
**************************************************
Training model for AIZ
Model for AIZ trained in 0.37 seconds
**************************************************
Training model for AJG
Model for AJG trained in 0.31 seconds
**************************************************
Training model for ALL
Model for ALL trained in 0.34 seconds
**************************************************
Training model for AMP
Model for AMP trained in 0.35 seconds
**************************************************
Training model for AON
Model for AON trained in 0.31 seconds
**************************************************
Training model for APO
Model for APO trained in 0.32 seconds
*****************************************

In [8]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes[ticker] = mape_
    maes[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ACGL, MAPE: 7.81, MAE: 7.75
**************************************************
Ticker: AFL, MAPE: 32.77, MAE: 34.09
**************************************************
Ticker: AIG, MAPE: 2.96, MAE: 2.34
**************************************************
Ticker: AIZ, MAPE: 18.79, MAE: 37.93
**************************************************
Ticker: AJG, MAPE: 15.83, MAE: 48.01
**************************************************
Ticker: ALL, MAPE: 10.08, MAE: 19.28
**************************************************
Ticker: AMP, MAPE: 13.20, MAE: 67.99
**************************************************
Ticker: AON, MAPE: 12.24, MAE: 45.14
**************************************************
Ticker: APO, MAPE: 16.88, MAE: 25.89
**************************************************
Ticker: AXP, MAPE: 14.37, MAE: 40.93
**************************************************
Ticker: BAC, MAPE: 12.76, MAE: 5.49
**************************************************
Ticker: BEN, MAPE: 13.99, MAE: 2.73


In [9]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(11.895948727266521, 24.677879089768687)

### Global model

In [10]:
print(f"Training global model")
start = time.time()
model = XGBModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    },
    use_static_covariates=True,
    n_estimators=1000,
    eta=0.01,
    max_depth=8,
    subsample=0.8
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 22.04 seconds
**************************************************


In [11]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes[ticker] = mape_
    maes[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for ACGL, MAPE: 2.28, MAE: 2.19
**************************************************
Global model for AFL, MAPE: 2.88, MAE: 2.94
**************************************************
Global model for AIG, MAPE: 2.04, MAE: 1.54
**************************************************
Global model for AIZ, MAPE: 1.91, MAE: 3.70
**************************************************
Global model for AJG, MAPE: 1.78, MAE: 5.24
**************************************************
Global model for ALL, MAPE: 1.76, MAE: 3.24
**************************************************
Global model for AMP, MAPE: 8.00, MAE: 39.88
**************************************************
Global model for AON, MAPE: 1.75, MAE: 6.09
**************************************************
Global model for APO, MAPE: 5.57, MAE: 7.41
**************************************************
Global model for AXP, MAPE: 2.01, MAE: 5.31
**************************************************
Global model for BAC, MAPE: 2.41, MAE: 0.98
****

In [12]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(3.1863993367901537, 8.485099203952252)

## Prophet model

In [13]:
models_prophet = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = Prophet(
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']  # automatically generate future covariates from time series
            }
        },
        seasonality_mode='multiplicative',
    )

    model.fit(
        train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_prophet[ticker] = model

Training model for ACGL


21:15:50 - cmdstanpy - INFO - Chain [1] start processing
21:15:53 - cmdstanpy - INFO - Chain [1] done processing


Model for ACGL trained in 3.03 seconds
**************************************************
Training model for AFL


21:15:53 - cmdstanpy - INFO - Chain [1] start processing
21:15:55 - cmdstanpy - INFO - Chain [1] done processing


Model for AFL trained in 2.45 seconds
**************************************************
Training model for AIG


...

Model for WFC trained in 3.13 seconds
**************************************************
Training model for WRB


21:19:05 - cmdstanpy - INFO - Chain [1] start processing
21:19:06 - cmdstanpy - INFO - Chain [1] done processing


Model for WRB trained in 2.09 seconds
**************************************************
Training model for WTW


21:19:07 - cmdstanpy - INFO - Chain [1] start processing
21:19:10 - cmdstanpy - INFO - Chain [1] done processing


Model for WTW trained in 3.20 seconds
**************************************************


In [14]:
mapes_prophet = {}
maes_prophet = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_prophet[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_prophet[ticker] = mape_
    maes_prophet[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ACGL, MAPE: 15.04, MAE: 14.68
**************************************************
Ticker: AFL, MAPE: 11.96, MAE: 12.46
**************************************************
Ticker: AIG, MAPE: 5.99, MAE: 4.55
**************************************************
Ticker: AIZ, MAPE: 11.32, MAE: 23.10
**************************************************
Ticker: AJG, MAPE: 6.90, MAE: 21.35
**************************************************
Ticker: ALL, MAPE: 22.15, MAE: 41.01
**************************************************
Ticker: AMP, MAPE: 11.69, MAE: 58.34
**************************************************
Ticker: AON, MAPE: 26.41, MAE: 97.52
**************************************************
Ticker: APO, MAPE: 10.73, MAE: 15.62
**************************************************
Ticker: AXP, MAPE: 23.32, MAE: 63.99
**************************************************
Ticker: BAC, MAPE: 28.48, MAE: 11.90
**************************************************
Ticker: BEN, MAPE: 7.54, MAE: 1.51

In [15]:
np.mean(list(mapes_prophet.values())), np.mean(list(maes_prophet.values()))

(20.8301610519168, 33.46482449457745)

## Theta model

In [16]:
models_theta = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoTheta(
        season_length=30
    )

    model.fit(
        series=train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_theta[ticker] = model

Training model for ACGL
Model for ACGL trained in 1.23 seconds
**************************************************
Training model for AFL
Model for AFL trained in 0.98 seconds
**************************************************
Training model for AIG
Model for AIG trained in 0.61 seconds
**************************************************
Training model for AIZ
Model for AIZ trained in 1.18 seconds
**************************************************
Training model for AJG
Model for AJG trained in 2.15 seconds
**************************************************
Training model for ALL
Model for ALL trained in 1.06 seconds
**************************************************
Training model for AMP
Model for AMP trained in 1.34 seconds
**************************************************
Training model for AON
Model for AON trained in 0.70 seconds
**************************************************
Training model for APO
Model for APO trained in 1.19 seconds
*****************************************

In [17]:
mapes_theta = {}
maes_theta = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_theta[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_theta[ticker] = mape_
    maes_theta[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ACGL, MAPE: 5.13, MAE: 5.09
**************************************************
Ticker: AFL, MAPE: 15.98, MAE: 16.88
**************************************************
Ticker: AIG, MAPE: 4.16, MAE: 3.15
**************************************************
Ticker: AIZ, MAPE: 10.13, MAE: 20.60
**************************************************
Ticker: AJG, MAPE: 14.51, MAE: 43.73
**************************************************
Ticker: ALL, MAPE: 8.01, MAE: 15.23
**************************************************
Ticker: AMP, MAPE: 12.30, MAE: 63.63
**************************************************
Ticker: AON, MAPE: 16.13, MAE: 59.08
**************************************************
Ticker: APO, MAPE: 15.06, MAE: 23.07
**************************************************
Ticker: AXP, MAPE: 10.76, MAE: 30.83
**************************************************
Ticker: BAC, MAPE: 10.01, MAE: 4.32
**************************************************
Ticker: BEN, MAPE: 7.76, MAE: 1.51
**

In [18]:
np.mean(list(mapes_theta.values())), np.mean(list(maes_theta.values()))

(11.853871508914272, 23.816626048663643)

## Linear Regression

### Local model

In [19]:
models_lr = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = LinearRegressionModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_lr[ticker] = model

Training model for ACGL
Model for ACGL trained in 0.13 seconds
**************************************************
Training model for AFL
Model for AFL trained in 0.12 seconds
**************************************************
Training model for AIG
Model for AIG trained in 0.12 seconds
**************************************************
Training model for AIZ
Model for AIZ trained in 0.12 seconds
**************************************************
Training model for AJG
Model for AJG trained in 0.12 seconds
**************************************************
Training model for ALL
Model for ALL trained in 0.12 seconds
**************************************************
Training model for AMP
Model for AMP trained in 0.12 seconds
**************************************************
Training model for AON
Model for AON trained in 0.12 seconds
**************************************************
Training model for APO
Model for APO trained in 0.12 seconds
*****************************************

In [20]:
mapes_lr = {}
maes_lr = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_lr[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_lr[ticker] = mape_
    maes_lr[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: ACGL, MAPE: 90.99, MAE: 84.84
**************************************************
Ticker: AFL, MAPE: 2.49, MAE: 2.57
**************************************************
Ticker: AIG, MAPE: 3.45, MAE: 2.67
**************************************************
Ticker: AIZ, MAPE: 3.90, MAE: 7.78
**************************************************
Ticker: AJG, MAPE: 10.15, MAE: 30.52
**************************************************
Ticker: ALL, MAPE: 118.23, MAE: 229.78
**************************************************
Ticker: AMP, MAPE: 2.92, MAE: 14.20
**************************************************
Ticker: AON, MAPE: 1.33, MAE: 4.66
**************************************************
Ticker: APO, MAPE: 5247.90, MAE: 7108.36
**************************************************
Ticker: AXP, MAPE: 5.96, MAE: 16.57
**************************************************
Ticker: BAC, MAPE: 2.13, MAE: 0.86
**************************************************
Ticker: BEN, MAPE: 96.84, MAE: 18.51


In [21]:
np.mean(list(mapes_lr.values())), np.mean(list(maes_lr.values()))

(1.0704046091313797e+19, 3.974853039434333e+19)

### Global model

In [22]:
print(f"Training global model")
start = time.time()
model = LinearRegressionModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    }
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 8.88 seconds
**************************************************


In [23]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes[ticker] = mape_
    maes[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for ACGL, MAPE: 4.94, MAE: 4.70
**************************************************
Global model for AFL, MAPE: 4.06, MAE: 4.17
**************************************************
Global model for AIG, MAPE: 4.16, MAE: 3.11
**************************************************
Global model for AIZ, MAPE: 3.72, MAE: 7.27
**************************************************
Global model for AJG, MAPE: 4.02, MAE: 11.88
**************************************************
Global model for ALL, MAPE: 3.21, MAE: 5.99
**************************************************
Global model for AMP, MAPE: 4.92, MAE: 24.32
**************************************************
Global model for AON, MAPE: 4.50, MAE: 15.92
**************************************************
Global model for APO, MAPE: 7.76, MAE: 10.81
**************************************************
Global model for AXP, MAPE: 4.84, MAE: 13.14
**************************************************
Global model for BAC, MAPE: 4.03, MAE: 1.61


In [24]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(4.930623811030288, 8.707990625977335)